# Car Price Prediction Project (Part I - Cleaning Data)

**Project Goal:** 

To develop a machine learning model capable of accurately predicting the price of cars based on their features.

**Notebook Objective:** 

This notebook focuses on the crucial first phase: data cleaning. The raw dataset, comprising approximately 29,480 car listings with 58 features, presents significant challenges typical of real-world data. Our primary goal here is to transform this raw data into a clean and structured format suitable for subsequent processes such as handling missing values and outlier detection.

**Key Challenges Identified:**

- Missing Data: Many columns suffer from a high percentage of null values, requiring careful consideration for imputation or removal.
- Incorrect Data Types: Numerous columns that represent numerical quantities (e.g., price, Mileage, Power, Engine size, Doors, Seats, CO₂-emissions) are stored as strings (object type). These will need parsing (extracting numbers, removing units like "km", "kW", "g/km") and conversion.
- Inconsistent Formatting: Textual data within columns might be inconsistent or require standardization.
- Complex Features: Columns like Comfort & Convenience, Entertainment & Media, Safety & Security, and Extras appear to contain lists of amenities as strings, which will need parsing (e.g., one-hot encoding or count features).
- Redundancy: Potential redundancy exists (e.g., make_model vs. make and model).
- Low Utility Columns: Columns with extremely few non-null values may offer little predictive power and could be candidates for removal.
- Date/Time Features: Columns like First registration, General inspection, Last service, and Production date need conversion to a usable format (e.g., age of car, time since last service).

**Cleaning Strategy Outline:**

This notebook will systematically address these challenges through steps including (but not limited to):
- Dropping columns with extremely low non-null counts or low relevance.
- Correcting data types:
    - Converting numerical data stored as strings to int or float.
    - Parsing units (e.g., "km", "kg", "kW", "HP", "cc", "€").

By the end of this notebook, we aim to have a significantly cleaner and more structured dataset, laying a solid foundation for handling missing values, detecting and imputing outliers and effective feature engineering.

In [224]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

In [225]:
df0 = pd.read_json("auto_scout_24.json")
df = df0.copy()

In [226]:
df.head()

,make_model,short_description,make,model,location,price,Body type,Type,Doors,Country version,Offer number,Warranty,Mileage,First registration,Gearbox,Fuel type,Colour,Paint,desc,seller,Seats,Power,Engine size,Gears,CO₂-emissions,Manufacturer colour,Drivetrain,Cylinders,Fuel consumption,\nComfort & Convenience\n,\nEntertainment & Media\n,\nSafety & Security\n,\nExtras\n,Empty weight,Model code,General inspection,Last service,Full service history,Non-smoker vehicle,Emission class,Emissions sticker,Upholstery colour,Upholstery,Production date,Previous owner,Other fuel types,Power consumption,Energy efficiency class,CO₂-efficiency,Fuel consumption (WLTP),CO₂-emissions (WLTP),Available from,Taxi or rental car,Availability,Last timing belt change,Electric Range (WLTP),Power consumption (WLTP),Battery Ownership
0,Mercedes-Benz A 160,CDi,\nMercedes-Benz\n,"[\n, A 160 ,\n]","P.I. EL PALMAR C/FORJA 6, 11500 PUERTO DE SAN...","€ 16,950.-","[\n, Compact, \n]","[\n, Used, \n]","[\n, 5, \n]","[\n, Spain, \n]","[\n, 7468157, \n]","[\n, 24 months, \n]","120,200 km",06/2016,[\nManual\n],Diesel,Black,Metallic,"[ , Precio al contado: 16950 euros, , , Vehí...",Dealer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mercedes-Benz EQE 350,350+,\nMercedes-Benz\n,"[\n, EQE 350 ,\n]","APARTADO DE CORREOS 1032, 26140 LOGROÑO, ES","€ 80,900.-","[\n, Compact, \n]","[\n, Pre-registered, \n]","[\n, 4, \n]","[\n, Spain, \n]","[\n, 7054863, \n]","[\n, 24 months, \n]","5,000 km",06/2022,[\nAutomatic\n],Electric,NaN,NaN,"[ , Precio al contado: 88900 euros, , AUTO OJ...",Dealer,"[\n, 5, \n]",[\n215 kW (292 hp)\n],[\n1 cc\n],[\n9\n],0 g/km (comb.),Negro Obsidiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mercedes-Benz A 45 AMG,S 4Matic+ 8G-DCT,\nMercedes-Benz\n,"[\n, A 45 AMG ,\n]","PORT. TARRACO, MOLL DE LLEVANT, Nº 5, LOC. 6-8...","€ 69,900.-","[\n, Compact, \n]","[\n, Used, \n]","[\n, 5, \n]","[\n, Spain, \n]","[\n, 7410454, \n]","[\n, 12 months, \n]","18,900 km",07/2020,[\nAutomatic\n],Gasoline,Grey,NaN,"[Gris MANUFAKTUR mountaingrau magno, , Linea/...",Dealer,"[\n, 5, \n]",[\n310 kW (421 hp)\n],"[\n1,991 cc\n]",[\n8\n],NaN,Mountaingrau magno,"[\n, 4WD, \n]",[\n4\n],"[[8.4 l/100 km (comb.)], [10.9 l/100 km (city)...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mercedes-Benz A 35 AMG,4Matic+ 7G-DCT,\nMercedes-Benz\n,"[\n, A 35 AMG ,\n]","Carrer de Provença, 31 Local, 8029 BARCELONA, ES","€ 46,990.-","[\n, Compact, \n]","[\n, Used, \n]","[\n, 5, \n]","[\n, Spain, \n]","[\n, 7464503, \n]","[\n, 12 months, \n]","18,995 km",01/2020,[\nAutomatic\n],Gasoline,Yellow,NaN,"[ , Precio al contado: 48990 euros, , , BONI...",Dealer,"[\n, 5, \n]",[\n225 kW (306 hp)\n],"[\n1,991 cc\n]",[\n7\n],NaN,Amarillo Sol,"[\n, 4WD, \n]",[\n4\n],"[[7.3 l/100 km (comb.)], [9.3 l/100 km (city)]...","[Air conditioning, Automatic climate control, ...",[USB],"[ABS, Central door lock, Driver-side airbag, E...",[Sport seats],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mercedes-Benz A 45 AMG,200CDI BE Line 4M 7G-DCT,\nMercedes-Benz\n,"[\n, A 45 AMG ,\n]","CARRIL ARAGONES 4, 30007 CASILLAS, ES","€ 16,800.-","[\n, Compact, \n]","[\n, Used, \n]","[\n, 5, \n]","[\n, Spain, \n]","[\n, 7127931, \n]","[\n, 12 months, \n]","197,000 km",09/2015,[\nAutomatic\n],Diesel,White,Metallic,"[ , Precio al contado: 17400 euros, , Vehícul...",Dealer,"[\n, 5, \n]",[\n100 kW (136 hp)\n],"[\n2,143 cc\n]",[\n7\n],NaN,NaN,"[\n, 4WD, \n]",[\n4\n],"[[4.9 l/100 km (comb.)], [5.7 l/100 km (city)]...","[Air conditioning, Electrical side mirrors, Mu...","[Bluetooth, CD player, MP3, USB]","[ABS, Central door lock, Driver-side airbag, E...",NaN,"[\n1,545 kg\n]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [227]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29480 entries, 0 to 29479
Data columns (total 58 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   make_model                28630 non-null  object 
 1   short_description         28630 non-null  object 
 2   make                      28630 non-null  object 
 3   model                     28630 non-null  object 
 4   location                  28630 non-null  object 
 5   price                     28630 non-null  object 
 6   Body type                 28630 non-null  object 
 7   Type                      28630 non-null  object 
 8   Doors                     28271 non-null  object 
 9   Country version           16889 non-null  object 
 10  Offer number              23100 non-null  object 
 11  Warranty                  15784 non-null  object 
 12  Mileage                   28629 non-null  object 
 13  First registration        28628 non-null  object 
 14  Gearbo

In [228]:
df.columns

Index(['make_model', 'short_description', 'make', 'model', 'location', 'price',
       'Body type', 'Type', 'Doors', 'Country version', 'Offer number',
       'Warranty', 'Mileage', 'First registration', 'Gearbox', 'Fuel type',
       'Colour', 'Paint', 'desc', 'seller', 'Seats', 'Power', 'Engine size',
       'Gears', 'CO₂-emissions', 'Manufacturer colour', 'Drivetrain',
       'Cylinders', 'Fuel consumption', '\nComfort & Convenience\n',
       '\nEntertainment & Media\n', '\nSafety & Security\n', '\nExtras\n',
       'Empty weight', 'Model code', 'General inspection', 'Last service',
       'Full service history', 'Non-smoker vehicle', 'Emission class',
       'Emissions sticker', 'Upholstery colour', 'Upholstery',
       'Production date', 'Previous owner', 'Other fuel types',
       'Power consumption', 'Energy efficiency class', 'CO₂-efficiency',
       'Fuel consumption (WLTP)', 'CO₂-emissions (WLTP)', 'Available from',
       'Taxi or rental car', 'Availability', 'Last timing b

In [229]:
new_columns = ['make_model', 'short_description', 'make', 'model', 'location', 'price',
       'body_type', 'type', 'doors', 'country_version', 'offer_number',
       'warranty', 'mileage', 'first_registration', 'gearbox', 'fuel_type',
       'color', 'paint', 'description', 'seller', 'seats', 'power', 'engine_size',
       'gears', 'co2_emissions', 'manufacturer_color', 'drivetrain',
       'cylinders', 'fuel_consumption', 'comfort_convenience',
       'entertainment_media', 'safety_security', 'extras',
       'empty_weight', 'model_code', 'general_inspection', 'last_service',
       'full_service_history', 'non_smoker_vehicle', 'emission_class',
       'emissions_sticker', 'upholstery_color', 'upholstery',
       'production_date', 'previous_owner', 'other_fuel_types',
       'power_consumption', 'energy_efficiency_class', 'co2_efficiency',
       'fuel_consumption_WLTP', 'co2_emissions_WLTP', 'available_from',
       'taxi_or_rental', 'availability', 'last_timing_belt_change',
       'electric_range_WLTP', 'power_consumption_WLTP',
       'battery_ownership']

df.columns = new_columns

In [230]:
null_percent = df.isnull().sum() / len(df) * 100
null_cols = null_percent[null_percent > 70].index

df.drop(null_cols, axis=1, inplace=True)

In [231]:
df.shape

(29480, 42)

In [232]:
df.sample(1)

,make_model,short_description,make,model,location,price,body_type,type,doors,country_version,offer_number,warranty,mileage,first_registration,gearbox,fuel_type,color,paint,description,seller,seats,power,engine_size,gears,co2_emissions,manufacturer_color,drivetrain,cylinders,fuel_consumption,comfort_convenience,entertainment_media,safety_security,extras,empty_weight,general_inspection,full_service_history,non_smoker_vehicle,emission_class,emissions_sticker,upholstery_color,upholstery,previous_owner
22213,Ford Mustang,GT Coupé 290PS/Premium-Paket 2/PDC,\nFord\n,"[\n, Mustang ,\n]","Feldbergstraße 1-7, 61231 Bad Nauheim, DE","€ 44,970.-","[\n, Coupe, \n]","[\n, Used, \n]","[\n, 3, \n]",NaN,"[\n, 84696, \n]",NaN,"32,993 km",09/2018,[\nManual\n],Super 95,Black,Metallic,"[SONDERAUSSTATTUNG:, schwarz / iridium schwarz...",Dealer,NaN,[\n213 kW (290 hp)\n],"[\n2,261 cc\n]",NaN,213 g/km (comb.),Iridium schwarz,NaN,[\n4\n],"[[9 l/100 km (comb.)], [12.4 l/100 km (city)],...","[Air conditioning, Automatic climate control, ...",[Radio],"[ABS, Adaptive Cruise Control, Alarm system, C...",[Catalytic Converter],NaN,09/2024,NaN,NaN,Euro 6,4 (Green),NaN,NaN,"[[32,993 km, 09/2018], 1]"


In [233]:
def anatomy_of(df, col):
    """
    Sketches the basics of a feature.
    """
    print('Value Type: ', type(df[col][494]))
    print('Number of null values: ', df[col].isnull().sum())
    print('Number of unique values: ', df[col].nunique())
    print('Top 10 most frequent values: \n', df[col].value_counts().head(10))

### make_model

In [235]:
anatomy_of(df, 'make_model')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  611
Top 10 most frequent values: 
 make_model
Renault Megane    863
SEAT Leon         787
Volvo V40         740
Dacia Sandero     730
Hyundai i30       706
Volvo C70         692
Opel Astra        675
Ford Mustang      663
Peugeot 308       537
Ford Focus        495
Name: count, dtype: int64


### short_description

In [237]:
anatomy_of(df, 'short_description')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  20947
Top 10 most frequent values: 
 short_description
                              213
D2 Momentum 120                88
D2 Kinetic 120                 87
Cabrio 1.4T S&S Excellence     85
Extreme+ 7-Sitzer TCe 110      57
SW 1.6 Multijet II Lounge      41
1.2 Lounge                     38
D3 Momentum Aut. 150           34
D4 R-Design Kinetic 190        34
D3 Kinetic 150                 34
Name: count, dtype: int64


In [238]:
blank_index = df[df.short_description == ''].index

df.loc[blank_index, 'short_description'] = np.nan

### make

In [240]:
anatomy_of(df, 'make')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  13
Top 10 most frequent values: 
 make
\nVolvo\n            3659
\nMercedes-Benz\n    2398
\nOpel\n             2385
\nPeugeot\n          2360
\nRenault\n          2351
\nFiat\n             2338
\nFord\n             2324
\nNissan\n           2064
\nToyota\n           2038
\nHyundai\n          1867
Name: count, dtype: int64


In [241]:
df.make[494]

'\nMercedes-Benz\n'

In [242]:
df['make'] = df.make.str.strip('\n')

In [243]:
df.make.value_counts()

make
Volvo            3659
Mercedes-Benz    2398
Opel             2385
Peugeot          2360
Renault          2351
Fiat             2338
Ford             2324
Nissan           2064
Toyota           2038
Hyundai          1867
SEAT             1743
Skoda            1566
Dacia            1537
Name: count, dtype: int64

### model

In [245]:
df.model[494]

['\n, SLK 200 ,\n']

In [246]:
df['model'] = df.model.map(lambda x: ",".join(x) if type(x) == list else x)

In [247]:
anatomy_of(df, 'model')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  594
Top 10 most frequent values: 
 model
\n, Megane ,\n     863
\n, Leon ,\n       787
\n, V40 ,\n        740
\n, Sandero ,\n    730
\n, i30 ,\n        706
\n, C70 ,\n        692
\n, Astra ,\n      675
\n, Mustang ,\n    663
\n, 308 ,\n        537
\n, Focus ,\n      495
Name: count, dtype: int64


In [248]:
df['model'] = df.model.str.strip('\n, ')

In [249]:
blank_index = df.model[df.model == ''].index
df.loc[blank_index, 'model'] = np.nan

In [250]:
df.model.isnull().sum()

1126

### location

In [252]:
anatomy_of(df, 'location')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  8181
Top 10 most frequent values: 
 location
Av. Laboral, 10,  28021 MADRID, ES                          306
Luckenwalder Berg 5,  14913 Jüterbog, DE                    170
Ctra. del Mig, 96,,  08097 L'Hospitalet de Llobregat, ES    146
9 boulevard Jules Ferry,  75011 Paris, FR                   142
Neuenhofstr. 77,  52078 Aachen, DE                          135
Bestel gemakkelijk Online 24/7,  1000 Brussel, BE           131
CARRETERA DE MATARO 10,  8903 SANT ADRIÀ DE BESÒS, ES       110
CARRETERA FUENCARRAL A HORTALEZA 89 D,  28050 MADRID, ES     98
Donnerwenge 2,  57439 Attendorn, DE                          97
calle la chaparrilla, 7,  41500 alcalá de guadaira, ES       95
Name: count, dtype: int64


In [253]:
df.location[494]

'Hofer Str. 45,  95632 Wunsiedel, DE'

### price

In [255]:
anatomy_of(df, 'price')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  5021
Top 10 most frequent values: 
 price
€ 14,990.-    222
€ 12,990.-    219
€ 16,990.-    186
€ 19,990.-    166
€ 9,990.-     160
€ 19,900.-    157
€ 18,990.-    156
€ 11,990.-    154
€ 14,900.-    141
€ 13,990.-    141
Name: count, dtype: int64


In [256]:
df['price'] = df.price.str.strip('€ .-').str.replace(',', '').astype('float')

### body_type

In [258]:
df['body_type'] = df.body_type.map(lambda x: ",".join(x) if type(x) == list else x)

In [259]:
anatomy_of(df, 'body_type')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  6
Top 10 most frequent values: 
 body_type
\n, Station wagon, \n       5448
\n, Off-Road/Pick-up, \n    5415
\n, Compact, \n             5387
\n, Sedan, \n               5043
\n, Coupe, \n               4009
\n, Convertible, \n         3328
Name: count, dtype: int64


In [260]:
df['body_type'] = df.body_type.str.strip(', \n')

### type

In [262]:
df['type'] = df.type.map(lambda x: ",".join(x) if type(x) == list else x)

In [263]:
anatomy_of(df, 'type')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  4
Top 10 most frequent values: 
 type
\n, Used, \n              25251
\n, Demonstration, \n      1433
\n, Pre-registered, \n     1377
\n, Employee's car, \n      569
Name: count, dtype: int64


In [264]:
df['type'] = df.type.str.strip(', \n')

### doors

In [266]:
df['doors'] = df.doors.map(lambda x: ",".join(x) if type(x) == list else x)

In [267]:
anatomy_of(df, 'doors')

Value Type:  <class 'str'>
Number of null values:  1209
Number of unique values:  6
Top 10 most frequent values: 
 doors
\n, 5, \n    17481
\n, 2, \n     5523
\n, 4, \n     3001
\n, 3, \n     2259
\n, 6, \n        5
\n, 1, \n        2
Name: count, dtype: int64


In [268]:
df['doors'] = df.doors.str.strip(', \n').astype('float')

### country_version

In [270]:
df['country_version'] = df.country_version.map(lambda x: ",".join(x) if type(x) == list else x)

In [308]:
anatomy_of(df, 'country_version')

Value Type:  <class 'str'>
Number of null values:  12591
Number of unique values:  25
Top 10 most frequent values: 
 country_version
\n, Germany, \n           7939
\n, Spain, \n             6376
\n, Italy, \n              679
\n, Belgium, \n            641
\n, European Union, \n     340
\n, Netherlands, \n        306
\n, Austria, \n            266
\n, France, \n             101
\n, United States, \n       57
\n, Czechia, \n             47
Name: count, dtype: int64


In [312]:
df['country_version'] = df.country_version.str.strip(', \n')

### offer_number

In [319]:
df['offer_number'] = df.offer_number.map(lambda x: ",".join(x) if type(x) == list else x)

In [321]:
anatomy_of(df, 'offer_number')

Value Type:  <class 'str'>
Number of null values:  6380
Number of unique values:  20945
Top 10 most frequent values: 
 offer_number
\n, 1, \n                 28
\n, L-Vorlauf 2023, \n    10
\n, 20, \n                 9
\n, 30, \n                 9
\n, RE82542, \n            8
\n, 12, \n                 8
\n, SE63327, \n            7
\n, PC57566, \n            7
\n, 10, \n                 7
\n, UC99209, \n            7
Name: count, dtype: int64


In [325]:
df['offer_number'] = df.offer_number.str.strip(', \n')

### warranty

In [334]:
df['warranty'] = df.warranty.map(lambda x: ",".join(x) if type(x) == list else x)

In [336]:
anatomy_of(df, 'warranty')

Value Type:  <class 'str'>
Number of null values:  13696
Number of unique values:  65
Top 10 most frequent values: 
 warranty
\n, 12 months, \n    9545
\n, Yes, \n          2319
\n, 24 months, \n    1515
\n, 60 months, \n     968
\n, 6 months, \n      398
\n, 36 months, \n     257
\n, 3 months, \n      185
\n, 0 months, \n      133
\n, 48 months, \n      78
\n, 84 months, \n      47
Name: count, dtype: int64


In [340]:
df['warranty'] = df.warranty.str.strip('\n, ')

### mileage

In [347]:
anatomy_of(df, 'mileage')

Value Type:  <class 'str'>
Number of null values:  851
Number of unique values:  14183
Top 10 most frequent values: 
 mileage
10 km         586
1 km          172
50 km         133
100 km        119
5,000 km      118
5 km          114
20 km         101
15 km          91
100,000 km     79
3,000 km       77
Name: count, dtype: int64


In [355]:
df['mileage'] = df.mileage.str.split(" ").map(lambda x: x[0] if type(x) == list else x)\
                .str.replace(',', '').astype('float')

### first_registration

In [360]:
anatomy_of(df, 'first_registration')

Value Type:  <class 'str'>
Number of null values:  852
Number of unique values:  655
Top 10 most frequent values: 
 first_registration
08/2022    454
06/2022    428
05/2019    420
06/2019    418
07/2019    416
07/2022    412
06/2018    396
01/2019    390
05/2022    375
03/2019    372
Name: count, dtype: int64


In [381]:
df['first_registration'] = pd.to_datetime(df['first_registration'], format='%m/%Y').dt.to_period('M')

### gearbox

In [371]:
df['gearbox'] = df.gearbox.map(lambda x: ",".join(x) if type(x) == list else x)

In [373]:
anatomy_of(df, 'gearbox')

Value Type:  <class 'str'>
Number of null values:  1098
Number of unique values:  3
Top 10 most frequent values: 
 gearbox
\nManual\n            17023
\nAutomatic\n         11287
\nSemi-automatic\n       72
Name: count, dtype: int64


In [377]:
df['gearbox'] = df.gearbox.str.strip('\n ')

### fuel_type

In [384]:
anatomy_of(df, 'fuel_type')

Value Type:  <class 'str'>
Number of null values:  2637
Number of unique values:  46
Top 10 most frequent values: 
 fuel_type
Gasoline                                8532
Diesel                                  5911
Super 95                                3557
Diesel (Particle filter)                2816
Regular/Benzine 91                      2065
Super E10 95                            1016
Regular/Benzine 91 (Particle filter)     555
Super 95 (Particle filter)               537
Super E10 95 (Particle filter)           324
Regular/Benzine E10 91                   278
Name: count, dtype: int64


### color

In [387]:
anatomy_of(df, 'color')

Value Type:  <class 'str'>
Number of null values:  2574
Number of unique values:  14
Top 10 most frequent values: 
 color
Black     6473
Grey      5998
White     5185
Blue      3478
Red       2242
Silver    1622
Green      450
Brown      437
Orange     288
Beige      278
Name: count, dtype: int64


### paint

In [390]:
anatomy_of(df, 'paint')

Value Type:  <class 'str'>
Number of null values:  14985
Number of unique values:  2
Top 10 most frequent values: 
 paint
Metallic     14494
Uni/basic        1
Name: count, dtype: int64


### description

In [395]:
df['description'] = df.description.map(lambda x: ",".join(x) if type(x) == list else x)

In [400]:
df.description[494]

' * AIRSCARF Kopfraumheizung für Fahrer und Beifahrer, * Navigation: Becker MAP PILOT Vollintegr. (herausnehmbar m. USB-Anschl.) Navigationsmodul m. 2D/3D-Kartendarst. der digitalis. Geb. Europas Unterbringung in Aufnahmeschale im Handschuhfach Anz. der Navigation im Displ. des Audio 20 CD Bedienung ü. zentr. Controller akust. Ausgabe der Fahrempfehlungen über die Fahrzeuglautspr. visuelle Anzeige der Fahrempfehlungen im Kombiinstrument autom. Stauberücksichtigung (TMC Pro) Adresseing. ü. Sprache Updates von Kartendaten, Fkt.-Softw. u. weiteren Features ü. PC und Internet, * Räder: 43,2 cm (17"") Leichtmetallräder 4fach im 10-Speichen-Design VA: 7,5 J x 17 ET 42, Reifengröße 225/45 R 17 HA: 8,5 J x 17 ET 36, Reifengröße 245/40 R 17, * ABS, * ADAPTIV BRAKE mit HOLD-Funktion, * ATTENTION ASSIST, * Abgasanlage zweiflutig mit Endrohrblende, * Ablagen SLK-Klasse, * Adaptives Bremslicht blinkend, * Airbag für Fahrer und Beifahrer, * Airbag: Headbags in den Türen, * Airbag: Sidebags in den Si

### seller

In [403]:
anatomy_of(df, 'seller')

Value Type:  <class 'str'>
Number of null values:  850
Number of unique values:  2
Top 10 most frequent values: 
 seller
Dealer            26318
Private seller     2312
Name: count, dtype: int64


In [409]:
df['seller'] = df.seller.str.title()

### seats

In [418]:
df['seats'] = df.seats.map(lambda x: ",".join(x) if type(x) == list else x)

In [422]:
anatomy_of(df, 'seats')

Value Type:  <class 'str'>
Number of null values:  3975
Number of unique values:  11
Top 10 most frequent values: 
 seats
\n, 5, \n    18308
\n, 4, \n     5390
\n, 2, \n     1186
\n, 7, \n      488
\n, 8, \n       43
\n, 9, \n       35
\n, 3, \n       25
\n, 6, \n       12
\n, 0, \n        9
\n, 1, \n        7
Name: count, dtype: int64


In [428]:
df['seats'] = df.seats.str.strip(', \n').astype('float')

### power

In [433]:
df['power'] = df.power.map(lambda x: ",".join(x) if type(x) == list else x)

In [435]:
anatomy_of(df, 'power')

Value Type:  <class 'str'>
Number of null values:  1422
Number of unique values:  351
Top 10 most frequent values: 
 power
\n110 kW (150 hp)\n    1992
\n96 kW (131 hp)\n     1356
\n88 kW (120 hp)\n     1182
\n81 kW (110 hp)\n     1166
\n66 kW (90 hp)\n      1110
\n103 kW (140 hp)\n    1033
\n85 kW (116 hp)\n     1002
\n74 kW (101 hp)\n      892
\n140 kW (190 hp)\n     787
\n51 kW (69 hp)\n       701
Name: count, dtype: int64


In [441]:
df['power'] = df.power.str.strip('\n')

### engine_size

In [448]:
df['engine_size'] = df.engine_size.map(lambda x: ",".join(x) if type(x) == list else x)

In [450]:
anatomy_of(df, 'engine_size')

Value Type:  <class 'str'>
Number of null values:  2253
Number of unique values:  467
Top 10 most frequent values: 
 engine_size
\n1,598 cc\n    2099
\n999 cc\n      2068
\n1,969 cc\n    1929
\n1,461 cc\n    1110
\n1,998 cc\n    1000
\n998 cc\n       925
\n1,199 cc\n     867
\n1,498 cc\n     782
\n1,997 cc\n     665
\n1,968 cc\n     625
Name: count, dtype: int64


In [458]:
df['engine_size'] = df.engine_size.str.strip(' cc\n').str.replace(',', '').astype('float')

### gears

In [463]:
df['gears'] = df.gears.map(lambda x: ",".join(x) if type(x) == list else x)

In [465]:
anatomy_of(df, 'gears')

Value Type:  <class 'str'>
Number of null values:  10526
Number of unique values:  11
Top 10 most frequent values: 
 gears
\n6\n     8412
\n5\n     5335
\n7\n     1738
\n8\n     1690
\n1\n      712
\n9\n      642
\n4\n      256
\n10\n     112
\n0\n       31
\n3\n       24
Name: count, dtype: int64


In [471]:
df['gears'] = df.gears.str.strip('\n').astype('float')

### co2_emissions

In [474]:
anatomy_of(df, 'co2_emissions')

Value Type:  <class 'str'>
Number of null values:  10886
Number of unique values:  347
Top 10 most frequent values: 
 co2_emissions
0 g/km (comb.)      1038
119 g/km (comb.)     393
124 g/km (comb.)     340
129 g/km (comb.)     319
114 g/km (comb.)     313
99 g/km (comb.)      312
115 g/km (comb.)     304
139 g/km (comb.)     298
109 g/km (comb.)     265
108 g/km (comb.)     262
Name: count, dtype: int64


### manufacturer_color

In [477]:
anatomy_of(df, 'manufacturer_color')

Value Type:  <class 'str'>
Number of null values:  7693
Number of unique values:  4963
Top 10 most frequent values: 
 manufacturer_color
Blanco            1235
Gris               671
Azul               552
Negro              546
Rojo               381
Grijs metallic     375
Zwart metallic     295
Zwart              232
Schwarz            228
Wit                223
Name: count, dtype: int64


### drivetrain

In [486]:
df['drivetrain'] = df.drivetrain.map(lambda x: ",".join(x) if type(x) == list else x)

In [488]:
anatomy_of(df, 'drivetrain')

Value Type:  <class 'str'>
Number of null values:  12587
Number of unique values:  3
Top 10 most frequent values: 
 drivetrain
\n, Front, \n    12066
\n, 4WD, \n       3252
\n, Rear, \n      1575
Name: count, dtype: int64


In [492]:
df['drivetrain'] = df.drivetrain.str.strip(', \n')

### cylinders

In [499]:
df['cylinders'] = df.cylinders.map(lambda x: ",".join(x) if type(x) == list else x)

In [501]:
anatomy_of(df, 'cylinders')

Value Type:  <class 'str'>
Number of null values:  10628
Number of unique values:  12
Top 10 most frequent values: 
 cylinders
\n4\n     13068
\n3\n      3258
\n6\n      1013
\n5\n       799
\n8\n       539
\n2\n       103
\n0\n        35
\n1\n        21
\n12\n        8
\n7\n         6
Name: count, dtype: int64


In [507]:
df['cylinders'] = df.cylinders.str.strip('\n').astype('float')

### fuel_consumption

In [516]:
import itertools

df['fuel_consumption'] = df.fuel_consumption.map(lambda x: ",".join(list(itertools.chain.from_iterable(x))) if type(x) == list else x)

In [518]:
anatomy_of(df, 'fuel_consumption')

Value Type:  <class 'str'>
Number of null values:  6095
Number of unique values:  3452
Top 10 most frequent values: 
 fuel_consumption
0 l/100 km (comb.)                                                 330
0 l/100 km (comb.),0 l/100 km (city),0 l/100 km (country)          306
3.4 l/100 km (comb.),3.7 l/100 km (city),3.2 l/100 km (country)    170
4 l/100 km (comb.)                                                 149
3.9 l/100 km (comb.)                                               133
3.8 l/100 km (comb.),4.2 l/100 km (city),3.6 l/100 km (country)    110
5.1 l/100 km (comb.),6.4 l/100 km (city),4.3 l/100 km (country)    105
5.4 l/100 km (comb.),6.2 l/100 km (city),5 l/100 km (country)      101
3.7 l/100 km (comb.),4.4 l/100 km (city),3.3 l/100 km (country)    101
4.5 l/100 km (comb.)                                                97
Name: count, dtype: int64


### comfort_convenience

In [529]:
df['comfort_convenience'] = df.comfort_convenience.map(lambda x: ",".join(x) if type(x) == list else x)

In [531]:
anatomy_of(df, 'comfort_convenience')

Value Type:  <class 'str'>
Number of null values:  4047
Number of unique values:  14438
Top 10 most frequent values: 
 comfort_convenience
Air conditioning, Automatic climate control, Electrical side mirrors, Multi-function steering wheel, Power windows                    244
Air conditioning, Automatic climate control, Cruise control                                                                           171
Air conditioning, Automatic climate control, Cruise control, Electrical side mirrors, Multi-function steering wheel, Power windows    168
Air conditioning                                                                                                                      157
Power windows                                                                                                                         150
Air conditioning, Power windows                                                                                                       146
Air conditioning, Automatic clima

### entertainment_media

In [536]:
df['entertainment_media'] = df.entertainment_media.map(lambda x: ",".join(x) if type(x) == list else x)

In [538]:
anatomy_of(df, 'entertainment_media')

Value Type:  <class 'str'>
Number of null values:  5836
Number of unique values:  1944
Top 10 most frequent values: 
 entertainment_media
Bluetooth, USB                                                                                                     753
Bluetooth                                                                                                          665
On-board computer                                                                                                  662
CD player, On-board computer, Radio                                                                                644
Radio                                                                                                              425
Android Auto, Apple CarPlay, Bluetooth, Digital radio, Hands-free equipment, On-board computer, Radio, USB         414
Bluetooth, CD player, Hands-free equipment, MP3, On-board computer, Radio, USB                                     408
CD player, Radio             

### safety_security

In [543]:
df['safety_security'] = df.safety_security.map(lambda x: ",".join(x) if type(x) == list else x)

In [545]:
anatomy_of(df, 'safety_security')

Value Type:  <class 'str'>
Number of null values:  4065
Number of unique values:  12549
Top 10 most frequent values: 
 safety_security
Isofix                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    380
ABS, Central door lock, Driver-side airbag, Electronic stability control, Immobilizer, Isofix, Passenger-side airbag, Power steering, Side airbag                                                                                  

### extras

In [552]:
df['extras'] = df.extras.map(lambda x: ",".join(x) if type(x) == list else x)

In [554]:
anatomy_of(df, 'extras')

Value Type:  <class 'str'>
Number of null values:  6000
Number of unique values:  9360
Top 10 most frequent values: 
 extras
Alloy wheels                                                      3408
Alloy wheels, Roof rack                                            380
Alloy wheels, Sport seats                                          372
Alloy wheels, Touch screen                                         231
Alloy wheels, Catalytic Converter, Touch screen                    178
Roof rack                                                          160
Alloy wheels, Sport seats, Sport suspension                        157
Alloy wheels, Catalytic Converter, Touch screen, Voice Control     153
Alloy wheels, Touch screen, Voice Control                          142
Alloy wheels (16")                                                 119
Name: count, dtype: int64


### empty_weight

In [563]:
df['empty_weight'] = df.empty_weight.map(lambda x: ",".join(x) if type(x) == list else x)

In [565]:
anatomy_of(df, 'empty_weight')

Value Type:  <class 'str'>
Number of null values:  11722
Number of unique values:  1218
Top 10 most frequent values: 
 empty_weight
\n1,395 kg\n    233
\n1,055 kg\n    224
\n1,423 kg\n    216
\n1,165 kg\n    200
\n1,320 kg\n    192
\n1,280 kg\n    179
\n1,090 kg\n    129
\n1,370 kg\n    120
\n1,615 kg\n    113
\n1,310 kg\n    112
Name: count, dtype: int64


In [573]:
df['empty_weight'] = df.empty_weight.str.strip(' kg\n').str.replace(',', '').astype('float')

### general_inspection

In [578]:
anatomy_of(df, 'general_inspection')

Value Type:  <class 'str'>
Number of null values:  17226
Number of unique values:  91
Top 10 most frequent values: 
 general_inspection
New        5883
05/2023     286
08/2023     280
03/2023     268
06/2023     266
04/2023     262
09/2023     255
07/2023     254
06/2024     216
05/2024     212
Name: count, dtype: int64


### full_service_history

In [583]:
anatomy_of(df, 'full_service_history')

Value Type:  <class 'str'>
Number of null values:  16915
Number of unique values:  1
Top 10 most frequent values: 
 full_service_history
Yes    12565
Name: count, dtype: int64


### non_smoker_vehicle

In [586]:
anatomy_of(df, 'non_smoker_vehicle')

Value Type:  <class 'str'>
Number of null values:  17886
Number of unique values:  1
Top 10 most frequent values: 
 non_smoker_vehicle
Yes    11594
Name: count, dtype: int64


### emission_class

In [589]:
anatomy_of(df, 'emission_class')

Value Type:  <class 'str'>
Number of null values:  11621
Number of unique values:  9
Top 10 most frequent values: 
 emission_class
Euro 6          6418
Euro 6d-TEMP    3399
Euro 6d         2858
Euro 5          2389
Euro 4          1743
Euro 3           523
Euro 2           217
Euro 1           172
Euro 6c          140
Name: count, dtype: int64


### emissions_sticker

In [592]:
anatomy_of(df, 'emissions_sticker')

Value Type:  <class 'str'>
Number of null values:  20066
Number of unique values:  4
Top 10 most frequent values: 
 emissions_sticker
4 (Green)         9230
1 (No sticker)     176
3 (Yellow)           6
2 (Red)              2
Name: count, dtype: int64


### upholstery_color

In [595]:
anatomy_of(df, 'upholstery_color')

Value Type:  <class 'str'>
Number of null values:  14911
Number of unique values:  11
Top 10 most frequent values: 
 upholstery_color
Black     10416
Grey       2038
Other      1003
Beige       466
Brown       275
Red         159
White        93
Blue         82
Orange       23
Green         8
Name: count, dtype: int64


### upholstery

In [598]:
anatomy_of(df, 'upholstery')

Value Type:  <class 'str'>
Number of null values:  10870
Number of unique values:  6
Top 10 most frequent values: 
 upholstery
Cloth           8736
Full leather    5439
Part leather    2835
alcantara        764
Other            628
Velour           208
Name: count, dtype: int64


In [602]:
df['upholstery'] = df.upholstery.str.title()

### previous_owner

In [617]:
df['previous_owner'] = df.previous_owner.map(lambda x: x[1] if type(x) == list else x).astype('float')

In [619]:
anatomy_of(df, 'previous_owner')

Value Type:  <class 'numpy.float64'>
Number of null values:  15465
Number of unique values:  13
Top 10 most frequent values: 
 previous_owner
1.0     9746
2.0     3221
3.0      699
4.0      184
5.0       69
6.0       37
7.0       22
8.0       16
9.0       14
12.0       3
Name: count, dtype: int64


### Saving Final Version

In [649]:
df.to_csv('car_project_clean.csv', index=False)